In [1]:
from openai import OpenAI
import gradio as gr

def submitText(text):
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": 
            """
                You are a Singaporean Navigation Assistant.\n
                The user will ask you to search for a route from the nearest designated type of public service location to a specified location.\n
                You must search the public service location that matches user's query, then show a route from that public service location to the specified location.\n
                You will provide a route by providing start point name and end point name.\n
                Your response must strictly be in following format:[StartPointName#EndPointName]\n
            """
            },
            {"role": "user", "content": text}
        ]
    )

    returnText=str(completion.choices[0].message.content)[1:-1]
    returns=returnText.split("#")

    returns[0]=returns[0].replace(" ","+")
    returns[1]=returns[1].replace(" ","+")

    returns[0]="[\""+returns[0]+",+Singapore\","
    returns[1]="\""+returns[1]+",+Singapore\"]"
    returnText=returns[0]+returns[1]
    print(returnText)
    return(gr.HTML("<iframe src=http://fuqianshan.asuscomm.com:5173?body="+returnText+" width=100%% height=1520px></iframe>"))

with gr.Blocks(title="BooleanPirates") as demo:
    with gr.Group():
        text=gr.Textbox(show_label=False)
        htmlBox=gr.HTML("<iframe src=http://fuqianshan.asuscomm.com:5173?body=[\"Fire+Station+Bukit+Merah,+Singapore\",\"SMU,+Singapore\"] width=100%% height=1520px></iframe>")

        text.submit(submitText,text,htmlBox)

demo.launch(server_name="0.0.0.0",server_port=7920)

input()

/Users/emmy/Desktop/Last_term_SMU/NUS_SMU/my_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://0.0.0.0:7920

To create a public link, set `share=True` in `launch()`.


["Chinatown+MRT+Station,+Singapore","Woodlands+MRT+Station,+Singapore"]


Traceback (most recent call last):
  File "/Users/emmy/Desktop/Last_term_SMU/NUS_SMU/my_venv/lib/python3.10/site-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/Users/emmy/Desktop/Last_term_SMU/NUS_SMU/my_venv/lib/python3.10/site-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/emmy/Desktop/Last_term_SMU/NUS_SMU/my_venv/lib/python3.10/site-packages/gradio/blocks.py", line 1786, in process_api
    result = await self.call_function(
  File "/Users/emmy/Desktop/Last_term_SMU/NUS_SMU/my_venv/lib/python3.10/site-packages/gradio/blocks.py", line 1338, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/emmy/Desktop/Last_term_SMU/NUS_SMU/my_venv/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/emmy/Desktop/Last_term_SMU

["Punggol+MRT+Station,+Singapore","National+Stadium,+Singapore"]
["Jurong+East+MRT+Station,+Singapore","Changi+Airport,+Singapore"]
["Policlinic+Tuas,+Singapore","Changi+Airport,+Singapore"]
["Woodlands+MRT+Station,+Singapore","East+Coast+Park,+Singapore"]


In [ ]:
#test